This example shows how to use hydrolinking methods to address a point to the NHDPlusV2.1 Medium Resolution.

For this example we use a point near the Red Cedar River in East Lansing, MI.

##### First set required parameters.
Note: The input parameters are the same for NHDPlusV2.1 and NHD High Resolution hydrolinking methods.

Required parameters include:

    *input_identifier: str, user supplied identifier
    *input_lat: float, latitude of the point to be hydrolinked
    *input_lon: float, longitude of the point to be hydrolinked
    
Other optional parameters include:

    *input_crs: int, coordinate reference system, default is 4269 (NAD83)
    *water_name: str, name of the water feature that the point is intended to be linked to
    *buffer_m = int, search distance in meters, default is 1000, max is 2000
    *method = str, link data to 'closest' or 'name_match' (closest having a matching water name), default 'name_match'
    *hydro_type = str, link to 'waterbody' or 'flowline' NHD features, default 'flowline'

In [1]:
#identifier -> number or string
input_identifier = 1
#latitude
input_lat = 42.7284
#longitude
input_lon = -84.5026
#coordinate system
input_crs = 4269  
#In this case we want the point to link to the Red Cedar River so water_name is a stream name
water_name = 'The Red Cedar River'
#buffer
buffer_m = 2000


method = 'name_match'
hydro_type = 'flowline'

First the module for National Hydrography Dataset Plus Version 2.1 is imported.  This HydroLink module is called nhdplusv2.

In [2]:
#Import hydrolink module for nhd_mr
from hydrolink import nhd_mr
import warnings; warnings.simplefilter('ignore')

In [3]:
# Initiate a Python object, this tries reprojecting data to nad83 and checks if data within bounding box of U.S.
hydrolink = nhd_mr.MedResPoint(input_identifier, input_lat, input_lon, water_name=water_name, input_crs=input_crs, buffer_m=buffer_m)

# Runs the HydroLink process using name_match method and flowline hydro_type.  
#This writes data out to a CSV in working directory named 'nhdplusv2_hydrolink_output.csv'
hydrolink.hydrolink_method(method=method, hydro_type=hydro_type)

In [4]:
#JSON representing all flowlines within buffer of the input coordinates (point to HydroLink)
hydrolink.flowlines_json

{'displayFieldName': 'GNIS_NAME',
 'hasM': True,
 'fieldAliases': {'GNIS_NAME': 'GNIS_Name',
  'LENGTHKM': 'LengthKm',
  'REACHCODE': 'ReachCode',
  'COMID': 'ComID',
  'TERMINALFLAG': 'TerminalFlag'},
 'geometryType': 'esriGeometryPolyline',
 'spatialReference': {'wkid': 4269, 'latestWkid': 4269},
 'fields': [{'name': 'GNIS_NAME',
   'type': 'esriFieldTypeString',
   'alias': 'GNIS_Name',
   'length': 65},
  {'name': 'LENGTHKM', 'type': 'esriFieldTypeDouble', 'alias': 'LengthKm'},
  {'name': 'REACHCODE',
   'type': 'esriFieldTypeString',
   'alias': 'ReachCode',
   'length': 14},
  {'name': 'COMID', 'type': 'esriFieldTypeInteger', 'alias': 'ComID'},
  {'name': 'TERMINALFLAG',
   'type': 'esriFieldTypeInteger',
   'alias': 'TerminalFlag'}],
 'features': [{'attributes': {'GNIS_NAME': 'Red Cedar River',
    'LENGTHKM': 8.42166733289811,
    'REACHCODE': '04050004000126',
    'COMID': 12242482,
    'TERMINALFLAG': 0},
   'geometry': {'hasM': True,
    'paths': [[[-84.4466485128023, 42.722

In [5]:
#HydroLink data for all flowlines within buffer, these data are evaluated to find closest flowline 
#with a matching name (using a fuzzy match approach)
hydrolink.flowlines_data

[{'gnis_name': 'Red Cedar River',
  'lengthkm': 8.42166733289811,
  'reachcode': '04050004000126',
  'comid': 12242482,
  'terminalflag': 0,
  'cleaned source water name': 'the red cedar river',
  'flowline name similarity': 0.8823529411764706,
  'flowline name similarity message': 'most likely match, based on fuzzy match',
  'meters from flowline': 44.33266059218964,
  'nhdplusv2 flowline measure': 28.850441143579545},
 {'gnis_name': 'Red Cedar River',
  'lengthkm': 0.790011225885614,
  'reachcode': '04050004000125',
  'comid': 12241734,
  'terminalflag': 0,
  'cleaned source water name': 'the red cedar river',
  'flowline name similarity': 0.8823529411764706,
  'flowline name similarity message': 'most likely match, based on fuzzy match',
  'meters from flowline': 1987.4230429025345,
  'nhdplusv2 flowline measure': 100.0},
 {'gnis_name': None,
  'lengthkm': 1.98386608634494,
  'reachcode': '04050004000960',
  'comid': 12241744,
  'terminalflag': 0,
  'flowline name similarity message

There are a few options for writing data to CSV files.  Both are set up with intentions of batch hydrolinking (multiple points -> see batch example notebook). 

Both methods accept an optional parameter for 'outfile_name'. 
    * default for write_reach_options() is 'mr_hydrolink_output.csv'
    * default for write_best() is 'mr__hydrolink_output.csv'

In [6]:
#HydroLink Data for the selected flowline.
#The name_match approach uses the closest flowline with a name match (meeting a specified similarity cutoff for names)
#If no flowline names (gnis_name) match the source water name, then the closest flowline is returned
hydrolink.hydrolink_flowline

{'nhdplusv2 flowline gnis name': 'Red Cedar River',
 'nhdplusv2 flowline length km': 8.42166733289811,
 'nhdplusv2 flowline reachcode': '04050004000126',
 'nhdplusv2 comid': 12242482,
 'nhdplusv2 terminal flag': 0,
 'cleaned source water name': 'the red cedar river',
 'flowline name similarity': 0.8823529411764706,
 'flowline name similarity message': 'most likely match, based on fuzzy match',
 'meters from flowline': 44.33266059218964,
 'nhdplusv2 flowline measure': 28.850441143579545,
 'closest flowline order': 1}

In [9]:
#View final data for returned reach
import pandas as pd
df = pd.read_csv('nhdplusv2_hydrolink_output.csv')
df.head()

,source id,source lat nad83,source lon nad83,source buffer meters,closest conluence meters,closest flowline order,total count flowlines in buffer,source water name,cleaned source water name,flowline name similarity,...,nhdplusv2 flowline reachcode,meters from flowline,nhdplusv2 flowline measure,nhdplusv2 terminal flag,nhdplusv2 waterbody permanent identifier,nhdplusv2 waterbody gnis name,nhdplusv2 waterbody reachcode,nhdplusv2 waterbody ftype,nhdplusv2 waterbody comid,hydrolink message
0,1,42.7284,-84.5026,2000,1987.423043,1,3,The Red Cedar River,the red cedar river,0.882353,...,4050004000126,44.332661,28.850441,0,NaN,NaN,NaN,NaN,NaN,NaN
